# Data Aggregation and Group Operations

After Loading, Merging, Preparing data set, we often need to compute group statistics or pivot tables for reporting or visualization.

**Overview**:
* Split pandas object into pieces 
* Computing group sumary statitics: count, mean, std, or user-defined functions.
* Applying function to each column of DataFrame
* Apply within-group transformations or other manipulations, like normalization,linear regression, rank, or subset selection
* Computing pivot tables
* Perform quantile analysis and other data-derived group analyses

# GroupBy Mechanics
As [Hadley Wickham](https://en.wikipedia.org/wiki/Hadley_Wickham) said, group is the same workflow: **split-apply-combine**
![split-apply-combine](https://image.slidesharecdn.com/slides-151008060416-lva1-app6892/95/pandas-powerful-data-analysis-tools-for-python-19-638.jpg?cb=1444284343)

Each grouping key can take many forms, and the keys do not have to be all of the sametype:
* A list or array of values that is the same length as the axis being grouped
* A value indicating a column name in a DataFrame
* A dict or Series giving a correspondence between the values on the axis being grouped and the group names
* A function to be invoked on the axis index or the individual labels in the index

In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np

In [2]:
df = DataFrame({
        'key1': ['a', 'a', 'b', 'b', 'a'],
        'key2': ['one', 'two', 'one', 'two', 'one'],
        'data1': np.random.rand(5),
        'data2': np.random.rand(5)
    })
df

,data1,data2,key1,key2
0,0.236779,0.839386,a,one
1,0.379287,0.520492,a,two
2,0.166433,0.733278,b,one
3,0.108102,0.281521,b,two
4,0.484072,0.664072,a,one


Suppose we want to compute the **mean** of data1 column by using groups labels from **key1**

In [23]:
grouped = df['data1'].groupby(df['key1'])

The importantthing here is that the data (a Series) has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the key1 column.

In [22]:
grouped.mean()

key1
a    0.366713
b    0.137267
Name: data1, dtype: float64

If instead we had passed multiple arrays as a list

In [14]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.329705
      two     0.747123
b     one     0.816156
      two     0.588410
Name: data1, dtype: float64

we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed. So we can **unstack hirrachical Series to get a DataFrame**

In [15]:
means.unstack()

key2,one,two
key1,,
a,0.329705,0.747123
b,0.816156,0.588410


We can not only group Series by Series key, but also group by an array

In [16]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2005, 2005, 2006])

In [19]:
df['data1'].groupby([states, years]).mean()

California  2005    0.781640
Ohio        2005    0.469020
            2006    0.309779
Name: data1, dtype: float64

Frequently the grouping information to be found in the same DataFrame as the data you want to work on

In [20]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.468844,0.624352
b,0.702283,0.265704


In [22]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.297860  0.232113
     two   0.065576  0.529084
b    one   0.084591  0.687881
     two   0.483881  0.437802

## Iterating Over Groups

> The GroupBy object supports iteration, generating a sequence of 2-tuples containing
the group name along with the chunk of data


In [21]:
df

,data1,data2,key1,key2
0,0.349631,0.840004,a,one
1,0.747123,0.908781,a,two
2,0.816156,0.248073,b,one
3,0.588410,0.283334,b,two
4,0.309779,0.124273,a,one


In [26]:
df.groupby('key1')

Groupby divide dataset in to chunks of data, and each of them can be access by **iterator**

In [28]:
for group1, group2  in df.groupby('key1'):
    print group1
    print group2

a
      data1     data2 key1 key2
0  0.349631  0.840004    a  one
1  0.747123  0.908781    a  two
4  0.309779  0.124273    a  one
b
      data1     data2 key1 key2
2  0.816156  0.248073    b  one
3  0.588410  0.283334    b  two


In [30]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.329705  0.482138
     two   0.747123  0.908781
b    one   0.816156  0.248073
     two   0.588410  0.283334

In the case of multiple keys

In [42]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print k1, k2
    print "==========="
    print group 
    print "==========="

a one
      data1     data2 key1 key2
0  0.349631  0.840004    a  one
4  0.309779  0.124273    a  one
a two
      data1     data2 key1 key2
1  0.747123  0.908781    a  two
b one
      data1     data2 key1 key2
2  0.816156  0.248073    b  one
b two
     data1     data2 key1 key2
3  0.58841  0.283334    b  two


Of course, we can choose to do whatever you want with the pieces of data. Such as computing a dict

In [43]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0  0.349631  0.840004    a  one
 1  0.747123  0.908781    a  two
 4  0.309779  0.124273    a  one, 'b':       data1     data2 key1 key2
 2  0.816156  0.248073    b  one
 3  0.588410  0.283334    b  two}

By default groupby groups on **axis=0**. We can change to groupby other axis

In [51]:
df

,data1,data2,key1,key2
0,0.349631,0.840004,a,one
1,0.747123,0.908781,a,two
2,0.816156,0.248073,b,one
3,0.588410,0.283334,b,two
4,0.309779,0.124273,a,one


In [50]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [52]:
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))

{dtype('float64'):       data1     data2
 0  0.349631  0.840004
 1  0.747123  0.908781
 2  0.816156  0.248073
 3  0.588410  0.283334
 4  0.309779  0.124273, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

## Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of
column names has the effect of selecting those columns for aggregation

In [4]:
df.groupby('key1')['data1']
df.groupby('key2')['data2']

In [7]:
df['data1'].groupby(df['key1'])
df['data2'].groupby(df['key2'])

With large dataset, we desire to aggregate only few columns. For example, we just want to compute **mean** of colums **data2**:

In [11]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.499721
     two   0.472313
b    one   0.382567
     two   0.255023

## Grouping with Dicts and Series

In [2]:
people = DataFrame(
    np.random.randn(5,5), 
    columns=['a','b','c','d','e'],
    index=['Joe', 'Steve', 'Wes', 'Jim','Travis']
)
people

,a,b,c,d,e
Joe,-1.021701,1.967837,-0.383938,0.552804,0.037949
Steve,0.852651,0.060948,-1.211080,-2.401445,-0.117531
Wes,-0.894143,0.468072,0.150973,-0.186364,-1.105413
Jim,0.309725,-0.100202,-0.563988,-0.308601,-0.478978
Travis,-0.624195,0.558411,0.152154,0.346468,1.071798


Add fre NaN value

In [3]:
people.ix[2:3, ['b', 'c']] = np.nan
people

,a,b,c,d,e
Joe,-1.021701,1.967837,-0.383938,0.552804,0.037949
Steve,0.852651,0.060948,-1.211080,-2.401445,-0.117531
Wes,-0.894143,NaN,NaN,-0.186364,-1.105413
Jim,0.309725,-0.100202,-0.563988,-0.308601,-0.478978
Travis,-0.624195,0.558411,0.152154,0.346468,1.071798


Sum together columns by group 

In [4]:
mapping = {
    'a': 'red', 
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red', 
    'f': 'orange'
}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

Now, you could easily construct an array from this dict to pass to groupby , but instead
we can just pass the dict:

In [5]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.168866,0.984085
Steve,-3.612525,0.796069
Wes,-0.186364,-1.999556
Jim,-0.872589,-0.269455
Travis,0.498622,1.006013


## Grouping with Functions

In [14]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.606119,1.867635,-0.947926,0.057839,-1.546442
5,0.852651,0.060948,-1.211080,-2.401445,-0.117531
6,-0.624195,0.558411,0.152154,0.346468,1.071798


In [7]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).sum()

a         b         c         d         e
3 one -1.915844  1.967837 -0.383938  0.366440 -1.067464
  two  0.309725 -0.100202 -0.563988 -0.308601 -0.478978
5 one  0.852651  0.060948 -1.211080 -2.401445 -0.117531
6 two -0.624195  0.558411  0.152154  0.346468  1.071798

## Grouping by Index Levels

A final convenience for hierarchically-indexed data sets is the ability to aggregate using
one of the levels of an axis index. To do this, pass the level number or name using the
**level** keyword:

In [17]:
columns = pd.MultiIndex.from_arrays([
        ['US', 'US', 'US', 'JP', 'JP'],
        [1, 3, 5, 1, 3]],
        names=['cty', 'tenor']      
    )
columns

MultiIndex(levels=[[u'JP', u'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=[u'cty', u'tenor'])

In [18]:
hier_df = DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      2.290035  0.287998 -0.411387 -1.599136 -0.860270
1      0.588556 -0.675430 -0.545894  1.995311  0.986947
2      0.160333 -1.355068  0.665241  0.092038  0.198539
3     -0.958391 -0.314045  0.519504  1.975757 -1.135650

# Data Aggregation

Data aggregation means produces scalar value from an array. Some of method we already known: **sum**, **mean**, **count**, **min**, **max**. But we still can use our own devising and aditionally.

In [23]:
df 

,data1,data2,key1,key2
0,0.005330,0.850351,a,one
1,0.383974,0.892172,a,two
2,0.602897,0.215213,b,one
3,0.924972,0.820692,b,two
4,0.852540,0.712074,a,one


In [24]:
grouped = df.groupby('key1')

In [30]:
grouped['data1'].quantile(0.9)

key1
a    0.758827
b    0.892765
Name: data1, dtype: float64

We can create our own aggregation method by pass method into parameter of **agg** function

In [31]:
def peak_to_peer(arr):
    return arr.max() - arr.min()

In [32]:
grouped.agg(peak_to_peer)

,data1,data2
key1,,
a,0.847210,0.180098
b,0.322076,0.605479


**notice**: **describe** is not an aggregation but it show the result of bunch of other aggs

In [33]:
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean   0.413948  0.818199
     std    0.424399  0.094256
     min    0.005330  0.712074
     25%    0.194652  0.781212
     50%    0.383974  0.850351
     75%    0.618257  0.871262
     max    0.852540  0.892172
b    count  2.000000  2.000000
     mean   0.763934  0.517952
     std    0.227742  0.428138
     min    0.602897  0.215213
     25%    0.683416  0.366582
     50%    0.763934  0.517952
     75%    0.844453  0.669322
     max    0.924972  0.820692